## PYHTON DB API

### PYODBC

In [ ]:
# !pip install pyodbc

In [1]:
import pyodbc

## SQL Server Authentication

In [2]:
def create_server_connection(driver, server, database, user, password):
    conn = None
    try: 
        conn = pyodbc.connect(DRIVER = driver, SERVER= server, DATABASE = database, UID= user, PWD= password)
        print("Successfull")
    except Error as err:
        print(f"Error:'{err}'")
    return conn

In [ ]:
# driver = '{ODBC Driver 17 for SQL Server}'
# server = 'localhost'
# database = 'master'
# user = 'sa'
# password = '----'
# conn = create_server_connection(driver, server, database, user, password)


In [3]:
pyodbc.drivers()

['SQL Server',
 'SQL Server Native Client 11.0',
 'SQL Server Native Client RDA 11.0',
 'ODBC Driver 17 for SQL Server',
 'Microsoft Access Driver (*.mdb, *.accdb)',
 'Microsoft Excel Driver (*.xls, *.xlsx, *.xlsm, *.xlsb)',
 'Microsoft Access Text Driver (*.txt, *.csv)']

## Windows Authentication

In [4]:
conn_string = "driver={ODBC Driver 17 for SQL Server}; server=localhost; database=master; TRUSTED_CONNECTION=yes;"

In [5]:
conn = pyodbc.connect(conn_string);

## Cursor process

In [6]:
crs = conn.cursor()

In [47]:
conn.autocommit = True

# Bu komut commit işleminin otomatik / sürekli yapılmasını sağlıyor

# sorgunun geçerli olabilmesi için sürekli conn.commit() yapmanın önüne geçildi.


In [8]:
conn.commit()

## Create Database

In [10]:
def create_database(conn, create_database_query): #parametreler: connection nesnesi ve query nesnesi
    crs = conn.cursor() #bir cursor nesnesi oluşturuyoruz. bunu query sonucundaki değerleri yakalamak için kullanacağız.
    try:
        crs.execute(create_database_query)#query'yi çalıştır sonucu yakala. queryleri excecute ile çalıştırıyoruz
        print("Database is Created Successfuly")
    except Error as err:
            print(f"Error:'{err}'")

In [14]:
create_database_query = 'CREATE DATABASE TestC'

In [15]:
create_database(conn, create_database_query)

Database is Created Successfuly


In [16]:
query = 'USE TestC'

In [17]:
crs.execute(query)

Aslında her defasında execute metodunun içine aşağıdaki gibi bildiğimiz SQL query si yazıyoruz. o da onu çalıştırıyor.

In [18]:
def execute_query(conn, query):
    crs = conn.cursor()
    try:
        crs.execute(query)
        conn.commit()
        print("Query Succeessful!")
    except Error as err:
        print(f"Error:'{err}'")

In [19]:
execute_query(conn, query)

Query Succeessful!


## Create Table

In [20]:
query = 'CREATE TABLE TestTable (\
           ID INT IDENTITY (1,1) NOT NULL, \
           FirstName VARCHAR(255) NOT NULL,\
           LastName VARCHAR(255) NOT NULL,\
           PRIMARY KEY(ID))'

In [21]:
execute_query(conn, query)

Query Succeessful!


## Insert

In [22]:
crs.execute("INSERT TestTable (FirstName, LastName) VALUES ('Bob', 'Marley')")

In [23]:
crs.execute("INSERT TestTable (FirstName, LastName) VALUES (?,?)", 'Bruce', 'Lee')

In [50]:
crs.execute("INSERT TestTable (FirstName, LastName) VALUES (?,?)",'Tom', 'Hanks')
crs.execute("INSERT TestTable (FirstName, LastName) VALUES (?,?)",'Bart', 'Simpson')

In [24]:
crs.execute("INSERT TestTable (FirstName, LastName) VALUES (?,?)",'Tom', 'Cat')
crs.execute("INSERT TestTable (FirstName, LastName) VALUES (?,?)",'Jerry', 'Mouse')
crs.execute("INSERT TestTable (FirstName, LastName) VALUES (?,?)",'Owen', 'William')
crs.execute("INSERT TestTable (FirstName, LastName) VALUES (?,?)",'Stefan', 'Müller')

## Delete

In [51]:
crs.execute("DELETE FROM TestTable WHERE FirstName = 'Jerry'")

In [52]:
crs.execute("DELETE FROM TestTable WHERE FirstName = ?" ,'Stefan')

## Rowcount

In [26]:
crs.rowcount

1

## Read data

**cursor.fetchall()**
bir sorgu sonucunun tüm satırlarını getirir. Tüm satırları bir tuple listesi olarak döndürür. Alınacak kayıt yoksa boş bir liste döndürülür.

**cursor.fetchmany(size)**
size değişkeni tarafından belirtilen satır sayısını döndürür. Tekrar tekrar çağrıldığında, bu yöntem bir sorgu sonucunun sonraki satır kümesini getirir ve bir tuple listesi döndürür.

**cursor.fetchone()**
tek bir kayıt döndürür. satır yoksa None döndürür. Başlangıçta ilk satırı, sonra her çalıştırıldığında bir sonraki satırı getiriyor.

## Fetchone

In [54]:
crs.execute('SELECT FirstName, LastName FROM TestTable')
row = crs.fetchone() # Her seferinde ayni tabloyu yeniden getirdiginden tablodaki ilk kaydi getirir
row

('Bob', 'Marley')

In [55]:
row = crs.fetchone() # tablodaki ilk kaydi getirir. Tekrar calisirsa siradaki satiri getirir
row

('Bruce', 'Lee')

In [56]:
row = crs.fetchone() # tablodaki ilk kaydi getirir. Tekrar calisirsa siradaki satiri getirir
row

('Tom', 'Cat')

In [57]:
row = crs.fetchone() # tablodaki ilk kaydi getirir. Tekrar calisirsa siradaki satiri getirir
row

('Owen', 'William')

## Fetchmany

In [58]:
crs.execute('SELECT FirstName, LastName FROM TestTable')
rows = crs.fetchmany(3)
rows

[('Bob', 'Marley'), ('Bruce', 'Lee'), ('Tom', 'Cat')]

## FetchAll

In [60]:
crs.execute('SELECT FirstName, LastName FROM TestTable')
rows = crs.fetchall()
rows

[('Bob', 'Marley'),
 ('Bruce', 'Lee'),
 ('Tom', 'Cat'),
 ('Owen', 'William'),
 ('Tom', 'Hanks'),
 ('Bart', 'Simpson')]

In [63]:
type(rows)

list

In [62]:
crs.execute("""SELECT ID, FirstName, LastName 
                FROM TestTable""")
rows = crs.fetchall()
rows

[(1, 'Bob', 'Marley'),
 (2, 'Bruce', 'Lee'),
 (3, 'Tom', 'Cat'),
 (5, 'Owen', 'William'),
 (7, 'Tom', 'Hanks'),
 (8, 'Bart', 'Simpson')]

In [64]:
for row in rows:
    print(row.ID, row.FirstName, row.LastName)

1 Bob Marley
2 Bruce Lee
3 Tom Cat
5 Owen William
7 Tom Hanks
8 Bart Simpson


In [66]:
crs.execute("""SELECT ID, FirstName, LastName 
               FROM TestTable
               WHERE FirstName = 'Owen' and LastName = 'William'""")
rows = crs.fetchall()
rows

[(5, 'Owen', 'William')]

In [67]:
# alternatif kullanım. WHERE satırına dikkat!

crs.execute("""SELECT ID, FirstName, LastName 
               FROM TestTable
               WHERE FirstName = ? and LastName = ?""", ['Owen', 'William'])
rows = crs.fetchall()
rows

[(5, 'Owen', 'William')]

## Fetchval()

SQL ifadeleri bir sonuç kümesi döndürürse, **ilk satırın ilk sütununu almak için** cursor nesnesinin **fetchval()** metodunu kullanırız. Sorgu boş bir sonuç kümesine sahipse, metod None değerini döndürür.

In [68]:
crs.execute("SELECT MAX(ID) FROM TestTable").fetchval()

8

In [69]:
crs.execute("SELECT FirstName FROM TestTable").fetchval()

'Bob'

# Pandas Methods
## Create DataFrame

In [74]:
import pandas as pd


In [75]:
def read_query (conn, query): #connection nesnesi yukarıda tanımlanmıştı.
    crs = conn. cursor()
    results = None
    crs.execute(query)
    results = crs.fetchall()
    return results

In [78]:
query = """SELECT ID, FirstName, LastName 
               FROM TestTable"""

In [79]:
result = read_query (conn, query)

In [80]:
result2 = crs.execute('SELECT * FROM TestTable')
result2

In [81]:
crs.execute('SELECT * FROM TestTable')
result = crs.fetchall()
result

[(1, 'Bob', 'Marley'),
 (2, 'Bruce', 'Lee'),
 (3, 'Tom', 'Cat'),
 (5, 'Owen', 'William'),
 (7, 'Tom', 'Hanks'),
 (8, 'Bart', 'Simpson')]

In [82]:
df = pd.DataFrame(result)

In [83]:
df

,0
0,"[1, Bob, Marley]"
1,"[2, Bruce, Lee]"
2,"[3, Tom, Cat]"
3,"[5, Owen, William]"
4,"[7, Tom, Hanks]"
5,"[8, Bart, Simpson]"


In [84]:
a = []

for result in result:
    result = list(result)
    a.append(result)

In [85]:
a

[[1, 'Bob', 'Marley'],
 [2, 'Bruce', 'Lee'],
 [3, 'Tom', 'Cat'],
 [5, 'Owen', 'William'],
 [7, 'Tom', 'Hanks'],
 [8, 'Bart', 'Simpson']]

In [86]:
columns = ['ID','FirstName', 'LastName']
df = pd.DataFrame(a, columns = columns)
df

,ID,FirstName,LastName
0,1,Bob,Marley
1,2,Bruce,Lee
2,3,Tom,Cat
3,5,Owen,William
4,7,Tom,Hanks
5,8,Bart,Simpson


In [87]:
query = 'USE SampleRetail'
execute_query(conn, query)

Query Succeessful!


In [88]:
df_prod = pd.read_sql("SELECT * FROM product.product", con =conn)
df_prod.head(8)

,product_id,product_name,brand_id,category_id,model_year,list_price
0,1,DENAQ - AC Adapter for TOSHIBA SATELLITE 1700 ...,23,4,2021,23.99
1,2,NS-SP1800BL 5.1-Channel Home Theater System (B...,5,5,2021,136.99
2,3,Acoustimass 6 Series V Home Theater Speaker Sy...,24,5,2021,599.00
3,4,Corsair Vengeance LPX 16GB (2x8GB) DDR4 DRAM 3...,6,4,2021,151.99
4,5,Details About Samsung Gear Iconx 2018 Edition ...,1,14,2021,199.99
5,6,2TB Red 5400 rpm SATA III 3.5 Internal NAS HDD,12,4,2021,89.95
6,7,Details About Alpine 480w 6.5 2way Typee Coaxi...,35,5,2021,59.99
7,8,Kenwood KFC-1653MRW 6.5 2-way Marine Speakers ...,31,5,2021,99.99


In [89]:
query = 'USE BikeStores'
execute_query(conn,query)

Query Succeessful!


In [90]:
crs.execute("SELECT * FROM production.brands").fetchall()

[(1, 'Electra'),
 (2, 'Haro'),
 (3, 'Heller'),
 (4, 'Pure Cycles'),
 (5, 'Ritchey'),
 (6, 'Strider'),
 (7, 'Sun Bicycles'),
 (8, 'Surly'),
 (9, 'Trek')]

In [91]:
pd.read_sql("SELECT * FROM production.brands", con = conn)

,brand_id,brand_name
0,1,Electra
1,2,Haro
2,3,Heller
3,4,Pure Cycles
4,5,Ritchey
5,6,Strider
6,7,Sun Bicycles
7,8,Surly
8,9,Trek


In [92]:
df_prod = pd.read_sql("SELECT * FROM production.products", con = conn)
df_prod.head(10)

,product_id,product_name,brand_id,category_id,model_year,list_price
0,1,Trek 820 - 2016,9,6,2016,379.99
1,2,Ritchey Timberwolf Frameset - 2016,5,6,2016,749.99
2,3,Surly Wednesday Frameset - 2016,8,6,2016,999.99
3,4,Trek Fuel EX 8 29 - 2016,9,6,2016,2899.99
4,5,Heller Shagamaw Frame - 2016,3,6,2016,1320.99
5,6,Surly Ice Cream Truck Frameset - 2016,8,6,2016,469.99
6,7,Trek Slash 8 27.5 - 2016,9,6,2016,3999.99
7,8,Trek Remedy 29 Carbon Frameset - 2016,9,6,2016,1799.99
8,9,Trek Conduit+ - 2016,9,5,2016,2999.99
9,10,Surly Straggler - 2016,8,4,2016,1549.00


## DF to Databese

In [93]:
import sqlalchemy
import urllib

In [ ]:
# driver = '{ODBC Driver 17 for SQL Server}'
# server = 'localhost' #şu anda server her birimizin kendi makinesi
# database = 'Test_1'#yeni bir db oluşturacağız
# user = 'sa' 
# password = '---'

In [95]:
# crs.execute('USE Test_1')

In [96]:
params = 'DRIVER='+driver + ';SERVER=' + server +';PORT=1433;DATABASE='+ database + ';UID=' + user + ';PWD=' + password 

NameError: name 'driver' is not defined

In [ ]:
db_params = urllib.parse.quote_plus (params)
engine = sqlalchemy.create_engine("mssql+pyodbc:///?odbc_connect={}".format(db_params))

In [ ]:
df_prod = pd.read_sql("SELECT * FROM production.products where product_id <10", con = conn)
df_prod.head(10)

In [ ]:
df_prod.to_sql("products_bikes", engine, index = False, if_exists = "append", schema = "dbo")